# Применение модели

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import json

In [2]:
# Вспомогательные функции
def clean_sentence(sent):
  sent = sent.split()
  clean_sent = list()
  for word in sent:
    word = word.lower()
    for letter in word:
      if not letter.isalpha() and not letter.isnumeric():
        word=word.replace(letter, "")
    clean_sent.append(word)
  return clean_sent

def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

In [3]:
# Загрузим словарь
with open('vocab.json', 'r') as f:
  vocab = json.load(f)

# Загрузим типы задач
with open('flipped_tasks.json', 'r') as f:
  flipped_tasks = json.load(f)

In [4]:
embed_len = 50
hidden_dim = 75
n_layers=1
class LSTMClassifier(nn.Module):
    def __init__(self):
        super(LSTMClassifier, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=len(vocab), embedding_dim=embed_len)
        self.lstm = nn.LSTM(input_size=embed_len, hidden_size=hidden_dim, num_layers=n_layers, batch_first=True)
        self.linear = nn.Linear(hidden_dim, len(flipped_tasks))

    def forward(self, X_batch):
        embeddings = self.embedding_layer(X_batch)
        hidden, carry = torch.randn(n_layers, len(X_batch), hidden_dim), torch.randn(n_layers, len(X_batch), hidden_dim)
        output, (hidden, carry) = self.lstm(embeddings, (hidden, carry))
        return self.linear(output[:,-1])

In [5]:
# Загрузим модель
model = torch.load("LSTM_classifier.pth")
model.eval()

LSTMClassifier(
  (embedding_layer): Embedding(3208, 50)
  (lstm): LSTM(50, 75, batch_first=True)
  (linear): Linear(in_features=75, out_features=2, bias=True)
)

In [6]:
example = 'Как установить срок выполнения задачи в WEEEK?'
def get_result(example):
  example = clean_sentence(example)
  input = prepare_sequence(example, vocab)
  out = model(input[None, :])
  pred = list(max(out)).index(max(max(out)).item())
  return flipped_tasks[str(pred)]

get_result(example)

'advice'

In [7]:
examples = ["Как организовать проект и добавиь в него задачки?",
"Как настроить автоматическое назначение задач на основе приоритета?",
"Как настроить интеграцию с системами учета рабочего времени?",
"Как оценить критические показатели и метрики для задач?",
"Как настроить интеграцию с облачными хранилищами файлов?",
"Как настроить доступ к задачам для внешних сторон?",
"Как оценить продвижение проекта по этапам и майлстонам?",
"Как оценить выполнение задачи с точки зрения качества?",
"Как настроить интеграцию с системами управления контентом?",
"Алексей Орлов, разработайте медиа-план на следующий год к 10 февраля.",
"Анна Кузнецова, разработайте анкету для исследования рынка к 5 апреля.",
"Александр Логинов, разработайте концепцию вирусной рекламной кампании к 1 июня.",
"Анатолий Петухов, проанализируйте рыночные возможности нового стартапа к 15 июля.",
"Алексей Жуков подготовьте предложения по оптимизации сайта к 20 августа.",
"Александр Логинов, спланируйте создание контент-плана на месяц вперед к 25 ноября.",
"Алексею Орлову нужно спланировать нашу работу по созданию аккаунтов компании в социальных сетях на следующий месяц к пятнице.",
"Анна Кузнецова, разработайте анкету для исследования потребителей и представьте варианты опросника завтра.",
"Александр должен разработать дизайн этикеток для новой продукции к 1 марта."]

In [8]:
for example in examples:
  print(f"{example}  =>  {get_result(example)}")

Как организовать проект и добавиь в него задачки?  =>  advice
Как настроить автоматическое назначение задач на основе приоритета?  =>  advice
Как настроить интеграцию с системами учета рабочего времени?  =>  advice
Как оценить критические показатели и метрики для задач?  =>  advice
Как настроить интеграцию с облачными хранилищами файлов?  =>  advice
Как настроить доступ к задачам для внешних сторон?  =>  advice
Как оценить продвижение проекта по этапам и майлстонам?  =>  advice
Как оценить выполнение задачи с точки зрения качества?  =>  advice
Как настроить интеграцию с системами управления контентом?  =>  advice
Алексей Орлов, разработайте медиа-план на следующий год к 10 февраля.  =>  task_create
Анна Кузнецова, разработайте анкету для исследования рынка к 5 апреля.  =>  task_create
Александр Логинов, разработайте концепцию вирусной рекламной кампании к 1 июня.  =>  task_create
Анатолий Петухов, проанализируйте рыночные возможности нового стартапа к 15 июля.  =>  task_create
Алексей 

In [9]:
# Не работает со словами, которых модель не видела в датасете

get_result("Кто такие Авоська и Небоська?")

KeyError: ignored